In [7]:
%load_ext autoreload
%autoreload 2
%pylab inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [8]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

In [9]:
sats_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/sats*')]
# sats_games = ['sats_2regions_2licenses_2players_5types']
# sats_games = ['sats_3regions_3licenses_5types']
# sats_games = ['sats_2regions_2licenses_2players_5types', 'sats_3regions_3licenses_5types']
# sats_games = ['rw_game_2_war_speedy_2']
# sats_games = ['confused_national']


# sats_games = ['confused_national', 'confused_national_with_signal', 'confused_national_with_undersell']
# sats_games = ['confused_national_with_signal', 'confused_national_with_undersell']
# sats_games = ['confused_national', 'confused_national_with_undersell']

sats_games = ['confused_national', 'regional_vs_national', 'regional_vs_national_signal']

# sats_games = ['regional_vs_national']
# sats_games = ['regional_vs_national_signal']


In [12]:
# SATS Games
# CONFIG = 'aug11'
# BR_CONFIG = 'oct31'
CONFIG = 'dec6'
BR_CONFIG = 'dec6br'
N_SEEDS = 3
# overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999999 --eval_zero false --total_timesteps 1_000_000 --dispatch_br false'
# NOTE: eval every is phrased in units of UPDATES, not steps
overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 2_500_000 --eval_zero false --total_timesteps 10_000_000 --dispatch_br true --br_overrides "--total_timesteps 2_000_000"'
for GAME_NAME in sats_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='new_configs_dec_8', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_require_br', game_name=f'{GAME_NAME}', overrides = overrides + ' --iterate_br false --require_br true', n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_random_ic', game_name=f'{GAME_NAME}', overrides = overrides + ' --random_ic true', n_seeds=N_SEEDS)

/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.game' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.experiment' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.register_model(new_class._meta.app_label, new_class)
/home/ubuntu/.pyenv/versions/3.8.2/envs/venv/lib/python3.8/site-packages/django/db/models/base.py:321: RuntimeWarning: Model 'auctions.equilibriumsolverrun' was already registered. Reloading models is not advised as it can lead to inconsistencies, most notably with related models.
  new_class._meta.apps.regis

Submitted batch job 68452
Submitted batch job 68453
Submitted batch job 68454
Submitted batch job 68455
Submitted batch job 68456
Submitted batch job 68457
Submitted batch job 68458
Submitted batch job 68459
Submitted batch job 68460
Submitted batch job 68461
Submitted batch job 68462
Submitted batch job 68463
Submitted batch job 68464
Submitted batch job 68465
Submitted batch job 68466
Submitted batch job 68467
Submitted batch job 68468
Submitted batch job 68469
Submitted batch job 68470
Submitted batch job 68471
Submitted batch job 68472
Submitted batch job 68473
Submitted batch job 68474
Submitted batch job 68475
Submitted batch job 68476
Submitted batch job 68477
Submitted batch job 68478
Submitted batch job 68479
Submitted batch job 68480
Submitted batch job 68481
Submitted batch job 68482
Submitted batch job 68483
Submitted batch job 68484
Submitted batch job 68485
Submitted batch job 68486
Submitted batch job 68487
Submitted batch job 68488
Submitted batch job 68489
Submitted ba

In [ ]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "straightforward" | awk '{print $1}' | xargs scancel

In [10]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)


In [ ]:
!squeue | wc -l

In [ ]:
# !scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

In [ ]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

In [ ]:
Game.objects.get(name='large_game_2_pricing_extreme.json').delete()

In [ ]:
!python /apps/open_spiel/web/auctions/manage.py nfsp --seed 109 --filename rounds.json --network_config_file mar17/lstm --experiment_name lstmtest --job_name "lstm_for_greg" --dispatch_br false --br_portfolio_path feb22 --eval_every 10_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --report_freq 10_000 --overwrite_db true

In [ ]:
list(Experiment.objects.all().values_list('name', flat=True))

In [ ]:
r = EquilibriumSolverRun.objects.get(name='large_game_3b_hide-mar17lstm-100')

In [ ]:
from auctions.webutils import *
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r)

In [ ]:
list(BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name'))

In [ ]:
BestResponse.objects.filter(checkpoint=pk)[0].brevaluation.__dict__

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r).values('best_response__name')

In [ ]:
BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name')

In [ ]:
experiments = ['mar24', 'mar22_4']
all_scripts = []
for e in experiments:
    scripts = glob.glob(f'/shared/outputs/{e}/*/evaluations/*.sh')
    scripts = [s for s in scripts if 'straightforward' not in s]
    scripts = [s for s in scripts if s[-5:] == "_2.sh"]
    all_scripts += scripts

In [ ]:
for script in all_scripts:
    parent = Path(script).parent
    !cd {parent} && sbatch {script}

In [ ]:
x = Evaluation.objects.last()

In [ ]:
x.samples['allocations']

In [ ]:
Evaluation.objects.last()

In [ ]:
import torch.nn.functional as F
import torch
x = torch.tensor([1,2,3])

In [ ]:
F.pad(x, (0, 2))

In [ ]:
torch.nn.functional.pad(input, pad, mode='constant', value=0.0)

In [ ]:
python /apps/open_spiel/web/auctions/manage.py nfsp --seed 100 --filename sats_5.json --network_config_file flatmlp/mlp --experiment_name test_require_br --job_name "sats_5_test-flatmlpmlp-100" --dispatch_br false --dry_run true --overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero false --num_training_episodes 8_000_000 --br_overrides "--num_training_episodes 1_000_000" --iterate_br false --require_br true

# Get env_and_model


# run_nfsp(env_and_model, 10_000, True, False, NullResultSaver(), 1, False, NullDispatcher(), eval_every, eval_every_early, eval_exactly, eval_zero, report_freq, dispatch_br, agent_selector, random_ic=False):

In [ ]:
BREvaluation.objects.filter(best_response__name='straightforward').last().expected_value_cdf

In [ ]:
!pip install wandb

In [ ]:
Evaluation.objects.last().samples.keys()

In [ ]:
Evaluation.objects.filter(
                    checkpoint__equilibrium_solver_run__experiment__name='testevolution3',
                    checkpoint__equilibrium_solver_run__generation=0,
                ).count()

In [ ]:
Evaluation.objects.last()

In [ ]:
!python /apps/cluster_open_spiel/web/auctions/manage.py ppo --filename bully1_2.json --network_config_file aug11/ppo --experiment_name bully1 --job_name "baseline" --overwrite_db True --eval_every 9999 --eval_zero false --total_timesteps 1_000_000 --use_wandb true --dry_run true

In [ ]:
!python /apps/cluster_open_spiel/web/auctions/manage.py ppo --filename bully_2.json --network_config_file aug11/ppo_potential_neg_pricing --experiment_name bully --job_name "neg_pricing" --overwrite_db True --eval_every 9999 --eval_zero false --total_timesteps 1_000_000 --use_wandb true --dry_run false

In [ ]:
Evaluation.objects.last()

In [ ]:
BestResponse.objects.exclude(name__contains='straightforward').last()

In [ ]:
potential_functions = ('auction_length', 'neg_auction_length', 'neg_revenue', 'revenue')

In [ ]:
from pathlib import Path
import glob
import yaml
ROOT = 'configs/nov23'
for f in glob.glob(f'{ROOT}/*.yml'):
    config = yaml.safe_load(open(f, 'rb'))
    stem = Path(f).stem
    for potential in potential_functions:
        config['potential_function'] = potential
        yaml.dump(config, open(f'{ROOT}/{stem}_{potential}.yml', 'w'))